<a href="https://colab.research.google.com/github/Alfred9/Natural-Language-Processing-Projects/blob/main/Text-Summarization/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text summarization is the process of creating shorter text without removing the semantic structure of text.

In [1]:
!pip install -q pyspark==3.3.0 spark-nlp==5.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.7/498.7 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.9 MB/s eta 0:00:00


In [2]:
import sparknlp

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd


spark = sparknlp.start()


In [3]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# Can take in document or sentence columns
bart = BartTransformer.pretrained(name="distilbart_xsum_12_6",lang='en') \
    .setInputCols('document')\
    .setOutputCol("Bart")\
    .setMaxOutputLength(100)

# Build pipeline with BART
pipe_components = [documentAssembler, bart]
pipeline = Pipeline().setStages( pipe_components)

distilbart_xsum_12_6 download started this may take some time.
Approximate size to download 699.7 MB
[OK!]


In [4]:
# Set the task for questions on T5
bart.setTask('summarize')

BartTRANSFORMER_41525e20b6b3

In [5]:
# https://www.reuters.com/article/instant-article/idCAKBN2AA2WF

text = """(Reuters) - Mastercard Inc said on Wednesday it was planning to offer support for some cryptocurrencies on its network this year, joining a string of big-ticket firms that have pledged similar support.

The credit-card giant’s announcement comes days after Elon Musk’s Tesla Inc revealed it had purchased $1.5 billion of bitcoin and would soon accept it as a form of payment.

Asset manager BlackRock Inc and payments companies Square and PayPal have also recently backed cryptocurrencies.

Mastercard already offers customers cards that allow people to transact using their cryptocurrencies, although without going through its network.

"Doing this work will create a lot more possibilities for shoppers and merchants, allowing them to transact in an entirely new form of payment. This change may open merchants up to new customers who are already flocking to digital assets," Mastercard said. (mstr.cd/3tLaPZM)

Mastercard specified that not all cryptocurrencies will be supported on its network, adding that many of the hundreds of digital assets in circulation still need to tighten their compliance measures.

Many cryptocurrencies have struggled to win the trust of mainstream investors and the general public due to their speculative nature and potential for money laundering.
"""
data = [[text]]
df=spark.createDataFrame(data).toDF('text')
#Predict on text data with BART
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['bart.result']).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                     |
+---------------------------------------------------------------------------------------------------------------------------+
|[The world's biggest credit- card company is joining the growing trend of accepting cryptocurrencies as a form of payment.]|
+---------------------------------------------------------------------------------------------------------------------------+



In [6]:
v = annotated_df.take(1)
print(f"Original Length {len(v[0].text)}   Summarized Length : {len(v[0].Bart[0].result)} ")

Original Length 1284   Summarized Length : 121 


In [7]:
# Full summarized text
v[0].Bart[0].result

"The world's biggest credit- card company is joining the growing trend of accepting cryptocurrencies as a form of payment."